In [59]:
from qaoa import *

In [60]:
# Example 1: Creating an Ising Hamiltonian and Finding Fisher Information Matrix

# Define the number of qubits and connectivity type ("open" or "closed")
n_qubits = 4
connectivity = "closed"

# Create an Ising Hamiltonian
ising_hamiltonian = H_zz_Ising(n_qubits, connectivity)


# Initialize QAOA with a specific depth and the Ising Hamiltonian
qaoa = QAOA(depth=1, H=ising_hamiltonian)

# Define a set of parameters
angles = [0.5, 0.2]  # Adjust these parameters as needed

# Calculate the Fisher Information Matrix
qfi_matrix = qaoa.qaoa_qfi_matrix(angles, state_ini=plus_state(n_qubits)) #|+>

print("Fisher Information Matrix:")
print(qfi_matrix)


Fisher Information Matrix:
[[1.60000000e+01 5.18565780e-16]
 [5.18565780e-16 2.92729239e+01]]


In [61]:
# Example 2: Creating a k-SAT Hamiltonian and Optimizing it with QAOA

# Define the number of variables (n), clauses (k), and alpha for k-SAT Hamiltonian
n = 5
k = 3
alpha = 2.0

# Create a k-SAT Hamiltonian
k_sat_hamiltonian = H_sat(n, k, alpha)

# Initialize QAOA with a specific depth and the k-SAT Hamiltonian
qaoa = QAOA(depth=2, H=k_sat_hamiltonian)


# Run the QAOA optimization using the L-BFGS-B method
qaoa.run_RI()

# Print the optimization results
print("QAOA Parameters:", qaoa.opt_angles)
print("Optimized QAOA Energy:", qaoa.q_energy)


QAOA Parameters: [5.38888091 5.57000923 0.4225245  3.14159265]
Optimized QAOA Energy: 0.3135798339235524


In [63]:
# Example 3: Solving the Max Cut Problem

# Define the graph adjacency matrix (replace this with your specific graph)
graph = [
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [1, 0, 0, 1],
    [0, 1, 1, 0]
]

# Number of qubits, which is the same as the number of nodes in the graph
num_qubits = len(graph)

# Convert the graph into the corresponding Ising Hamiltonian
max_cut_hamiltonian = Graph_to_Hamiltonian(graph, num_qubits)

# Now you can use your QAOA solver to find the maximum cut
qaoa = QAOA(depth=4, H=max_cut_hamiltonian)
qaoa.run_RI()

# Extract the results
max_cut_solution = qaoa.f_state  # The quantum state
max_cut_value = qaoa.q_energy  # The value of the maximum cut

# Output the results
print("Max Cut Solution (quantum state):\n", max_cut_solution)
print("Max Cut Value:", max_cut_value)

Max Cut Solution (quantum state):
 [4.89948797e-14-2.03163215e-12j 7.18245553e-08+6.32095511e-07j
 7.18245554e-08+6.32095511e-07j 8.35095282e-07+2.77767066e-07j
 7.18245553e-08+6.32095511e-07j 8.35095282e-07+2.77767066e-07j
 1.22178929e-01-6.96471327e-01j 7.18245552e-08+6.32095511e-07j
 7.18245552e-08+6.32095511e-07j 1.22178929e-01-6.96471327e-01j
 8.35095282e-07+2.77767066e-07j 7.18245553e-08+6.32095511e-07j
 8.35095282e-07+2.77767066e-07j 7.18245554e-08+6.32095511e-07j
 7.18245553e-08+6.32095511e-07j 4.89948797e-14-2.03163215e-12j]
Max Cut Value: -3.9999999999746545
